In [3]:
import requests

def chat_with_ollama(prompt, model="llama3.2:latest"):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False  # disable streaming
    }
    response = requests.post(url, json=payload)
    data = response.json()
    return data["response"]

if __name__ == "__main__":
    user_input = "Explain me string theory in simple terms."
    reply = chat_with_ollama(user_input)
    print(f"Ollama: {reply}")

KeyError: 'response'

In [6]:
import requests
import json

def chat_with_ollama(prompt, model="llama3.2:latest"):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": True  # enable streaming
    }

    with requests.post(url, json=payload, stream=True) as resp:
        for line in resp.iter_lines():
            if line:  # skip empty lines
                data = json.loads(line.decode("utf-8"))
                if "response" in data:
                    yield data["response"]
                if data.get("done", False):
                    break

if __name__ == "__main__":
    user_input = "Tell me about Golden Cross in trading."
    print("Ollama:", end=" ", flush=True)
    for chunk in chat_with_ollama(user_input):
        print(chunk, end="", flush=True)
    print()

Ollama: The Golden Cross is a technical analysis indicator used to identify the potential start of a new uptrend in a stock's price. It occurs when the 50-day moving average (MA) crosses above the 200-day MA, indicating that the shorter-term trend has shifted into the long-term trend.

Here are some key aspects of the Golden Cross:

1. **Signal generation**: The Golden Cross is generated when the 50-day MA crosses above the 200-day MA.
2. **Confirmation**: A buy signal is confirmed when the stock price closes above its 50-day MA after the cross-over.
3. **Mean reversion**: Some traders believe that a Golden Cross can also be a sign of mean reversion, as it indicates that the short-term trend may reverse and become more aligned with the long-term trend.
4. **Uptrend confirmation**: The Golden Cross is considered a bullish indicator, confirming an uptrend and indicating that the stock's price is likely to continue rising.

**When to use:**

1. **Buy signals**: Use the Golden Cross as a b

In [1]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

# 1. Define your LLM (Ollama backend)
llm = Ollama(model="llama3.2:latest", base_url="http://localhost:11434")  
# If running in K8s with ingress: base_url="http://your-ollama-service"

# 2. Chain 1: Generate outline
prompt1 = PromptTemplate(
    input_variables=["subject"],
    template="Create a 3-point outline for a blog on {subject}."
)
chain1 = LLMChain(llm=llm, prompt=prompt1)

# 3. Chain 2: Expand outline into a blog
prompt2 = PromptTemplate(
    input_variables=["outline"],
    template="Write a blog article based on this outline:\n{outline}"
)
chain2 = LLMChain(llm=llm, prompt=prompt2)

# 4. Combine into sequential chain
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

# 5. Run the chain
print(overall_chain.invoke("Kubernetes autoscaling"))

/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_32212/2714210623.py:6: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2:latest", base_url="http://localhost:11434")
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_32212/2714210623.py:14: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain1 = LLMChain(llm=llm, prompt=prompt1)




> Entering new SimpleSequentialChain chain...
Here is a 3-point outline for a blog on Kubernetes autoscaling:

I. Introduction to Kubernetes Autoscaling

* Brief overview of Kubernetes and its benefits
* Explanation of what Kubernetes autoscaling is and its importance in modern cloud-native applications
* Discussion of the challenges and limitations of traditional scaling approaches (e.g. static scaling, dynamic scaling)

II. Types of Kubernetes Autoscaling Strategies

* Horizontal Pod Autoscaling (HPA): a popular approach to automatically scale the number of replicas based on CPU utilization or other metrics
* Vertical Pod Autoscaling: a strategy for dynamically adjusting resources allocated to individual pods based on their workload needs
* Batch-based Autoscaling: an approach that scales applications in batches, often used for applications with predictable and recurring workloads

III. Best Practices for Implementing Kubernetes Autoscaling

* Overview of key considerations when im